## Fake News Classifier Using LSTM
Dataset: from kaggle

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/Data set/fake_news_dataset.csv", error_bad_lines=False, engine='python')

<ipython-input-2-c8ac160d6662>:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv("/content/drive/MyDrive/Data set/fake_news_dataset.csv", error_bad_lines=False, engine='python')
Skipping line 13535: field larger than field limit (131072)
Skipping line 19783: field larger than field limit (131072)
Skipping line 13541: Expected 5 fields in line 13541, saw 6
Skipping line 19782: Expected 5 fields in line 19782, saw 7
Skipping line 19783: Expected 5 fields in line 19783, saw 10
Skipping line 19785: Expected 5 fields in line 19785, saw 14
Skipping line 19787: Expected 5 fields in line 19787, saw 8
Skipping line 19789: Expected 5 fields in line 19789, saw 7
Skipping line 19790: Expected 5 fields in line 19790, saw 15
Skipping line 19791: Expected 5 fields in line 19791, saw 9
Skipping line 19792: Expected 5 fields in line 19792, saw 9
Skipping line 19793: Expected 5 fields in line 1

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


Here we have title of the news, the author name and the news article text. The label = 1 means that the news is fake and the label = 0 means the news is not fake.

In [4]:
df.shape

(20822, 5)

In [5]:
df.isnull().sum()

id           0
title      565
author    1975
text        59
label       22
dtype: int64

Dropping null values.

In [6]:
df = df.dropna()

In [7]:
df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

#### Getting the independent features

In [8]:
X = df.drop('label',axis=1)

In [9]:
y = df['label']

In [10]:
print(X.shape)
print(y.shape)

(18285, 4)
(18285,)


In [11]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [12]:
#Vocabulary size - lets consider we have 5000 unique words we have in our dictionary
voc_size = 5000

In [13]:
messages = X.copy()

In [14]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [15]:
messages.reset_index(inplace=True)

In [16]:
messages

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...,...
18280,20817,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
18281,20818,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
18282,20819,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
18283,20820,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [17]:
import nltk
import re
from nltk.corpus import stopwords

In [18]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [19]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0,len(messages)):
  review = re.sub('[^a-zA-Z]',' ',messages['title'][i])
  review = review.lower()
  review = review.split()

  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

In [32]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [20]:
onehot_repr = [one_hot(words, voc_size) for words in corpus]
onehot_repr

[[3911, 1040, 392, 1584, 110, 3989, 1894, 2277, 2057, 4306],
 [2487, 3188, 4241, 4673, 3389, 2100, 3759],
 [4121, 2284, 642, 4782],
 [2355, 1199, 1167, 301, 380, 2567],
 [660, 3389, 3347, 1821, 4254, 2598, 3389, 1638, 984, 3903],
 [863,
  1354,
  2238,
  202,
  3639,
  2401,
  2219,
  3023,
  3317,
  1505,
  792,
  971,
  4346,
  2461,
  3759],
 [2203, 4443, 504, 2355, 1164, 2349, 549, 4546, 2991, 3842, 2305],
 [863, 801, 558, 3394, 1002, 2607, 2401, 4362, 2991, 3842, 2305],
 [3173, 3573, 3084, 3876, 2975, 1120, 3907, 1906, 2401, 4472],
 [1327, 234, 998, 2315, 234, 2971, 1493, 3641],
 [2535, 3675, 3018, 1073, 2878, 3837, 4017, 2037, 2100, 2655, 370],
 [301, 129, 110, 1120, 2401, 1002],
 [4577, 4640, 2970, 1050, 1085, 99, 1730, 1329, 4482],
 [2536, 478, 468, 4752, 1499, 169, 4036, 2991, 3842, 2305],
 [4631, 1226, 2558, 1153, 2253, 2991, 3842, 2305],
 [1165, 455, 4713, 1994, 2393, 3729, 4447, 1106, 4599, 1690],
 [3521, 3654, 3188],
 [3401, 3418, 3009, 2751, 2401, 16, 2253, 3759],
 [3238,

### Embedding Representation

In [33]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=sent_length)
print(embedded_docs)

[[3911 1040  392 ...    0    0    0]
 [2487 3188 4241 ...    0    0    0]
 [4121 2284  642 ...    0    0    0]
 ...
 [1604 3016 3274 ...    0    0    0]
 [1038 1002  516 ...    0    0    0]
 [4670 3357 1925 ...    0    0    0]]


### Creating model

In [59]:
## Creating model
embedding_vector_features=50 ##features representation
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 20, 40)            200000    
                                                                 
 lstm_2 (LSTM)               (None, 200)               192800    
                                                                 
 dense_2 (Dense)             (None, 1)                 201       
                                                                 
Total params: 393001 (1.50 MB)
Trainable params: 393001 (1.50 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [35]:
len(embedded_docs),y.shape

(18285, (18285,))

In [50]:
import numpy as np
X_final=np.array(embedded_docs)
y_final = np.where(y == '1', 1.0, 0.0).astype(np.float32)

In [52]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [60]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 11s 43ms/step - loss: 0.3085 - accuracy: 0.8593 - val_loss: 0.2009 - val_accuracy: 0.9162
Epoch 2/10
192/192 [==============================] - 2s 13ms/step - loss: 0.1420 - accuracy: 0.9458 - val_loss: 0.2163 - val_accuracy: 0.9162
Epoch 3/10
192/192 [==============================] - 3s 14ms/step - loss: 0.1019 - accuracy: 0.9642 - val_loss: 0.2293 - val_accuracy: 0.9112
Epoch 4/10
192/192 [==============================] - 3s 13ms/step - loss: 0.0804 - accuracy: 0.9727 - val_loss: 0.3336 - val_accuracy: 0.9041
Epoch 5/10
192/192 [==============================] - 2s 10ms/step - loss: 0.0502 - accuracy: 0.9838 - val_loss: 0.4407 - val_accuracy: 0.8971
Epoch 6/10
192/192 [==============================] - 2s 10ms/step - loss: 0.0419 - accuracy: 0.9879 - val_loss: 0.3782 - val_accuracy: 0.9044
Epoch 7/10
192/192 [==============================] - 1s 8ms/step - loss: 0.0376 - accuracy: 0.9878 - val_loss: 0.4181 - val_accuracy: 0.9080

### Adding Dropout

In [66]:
from tensorflow.keras.layers import Dropout

embedding_vector_features = 40
model=Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])

In [67]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 12s 39ms/step - loss: 0.3104 - accuracy: 0.8539 - val_loss: 0.2034 - val_accuracy: 0.9153
Epoch 2/10
192/192 [==============================] - 3s 17ms/step - loss: 0.1480 - accuracy: 0.9443 - val_loss: 0.2116 - val_accuracy: 0.9163
Epoch 3/10
192/192 [==============================] - 2s 11ms/step - loss: 0.1106 - accuracy: 0.9599 - val_loss: 0.2644 - val_accuracy: 0.9127
Epoch 4/10
192/192 [==============================] - 2s 12ms/step - loss: 0.0886 - accuracy: 0.9691 - val_loss: 0.2591 - val_accuracy: 0.9120
Epoch 5/10
192/192 [==============================] - 2s 10ms/step - loss: 0.0692 - accuracy: 0.9763 - val_loss: 0.3041 - val_accuracy: 0.9137
Epoch 6/10
192/192 [==============================] - 1s 7ms/step - loss: 0.0534 - accuracy: 0.9810 - val_loss: 0.3574 - val_accuracy: 0.9022
Epoch 7/10
192/192 [==============================] - 2s 9ms/step - loss: 0.0427 - accuracy: 0.9849 - val_loss: 0.2657 - val_accuracy: 0.9117


### Performance Metrics and Accuracy


In [68]:
y_pred=model.predict(X_test)

189/189 [==============================] - 1s 3ms/step


In [69]:
y_pred=np.where(y_pred>0.5, 1,0)

In [70]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[3167,  262],
       [ 327, 2279]])

In [71]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.9024026512013256

In [72]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.91      0.92      0.91      3429
         1.0       0.90      0.87      0.89      2606

    accuracy                           0.90      6035
   macro avg       0.90      0.90      0.90      6035
weighted avg       0.90      0.90      0.90      6035

